# Importation nécessaire
il faut installer : 
pip install tensorflow== 1.14,
pip install contractions,
importer FrequentWords_v160320.csv
importer aussi no_not_clean.csv
importer aussi yes_not_clean.csv

In [9]:
#pip install tensorflow== 1.14
#pip install contractions
import re
import pandas as pd
import string
import numpy as np
import contractions #pip install 
import matplotlib.pyplot as plt


import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
nltk.download('all')

stop_words = stopwords.words('english')
porter = PorterStemmer()

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.python.keras.layers import Dropout,Dense
from tensorflow.python.keras import Sequential
from sklearn.metrics import confusion_matrix
#from keras.models import Sequential
#from keras.layers import Dense,  Activation
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy 
from sklearn import metrics
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers import Dense,  Activation

#from tensorflow.python.keras.layers import Dense, Activation
#from tensorflow.python.keras import Sequential

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_

# Fonctions Utils

In [0]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
from collections import defaultdict
token=ToktokTokenizer()
lemma=WordNetLemmatizer()

def remove_copyright(text):
  start0 = text.find("Copyright")
  start1 = text.find("©")
  if start0!=-1 & start1!=-1:
    start = min(start0, start1)
  elif start0!=-1 & start1==-1:
    start = start0
  elif start0==-1 & start1!=-1:
    start = start1
  else:
    start = -1
  if start!=-1: 
    text = text[0:start]
    
  return text

def remove_trialregistration(text):
  # The format for the ClinicalTrials.gov registry number is “NCT” followed by an 8-digit number
  text = re.sub('[N][C][T][[\d]{8}', '', text)
  text = re.sub('ClinicalTrials.gov', '', text)
  text = re.sub('Trial registration', '', text)
  text = re.sub('Trial register', '', text)
  text = re.sub('Identifier', '', text)
  
  return text

def remove_URL(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',str(text))
  

def classic_processing(text):
  text = remove_URL(text)
  text = remove_copyright(text)
  text = remove_trialregistration(text)
  text = text.translate(str.maketrans('', '', '/"()%-$@{}>=<[].;,:!?+&/#°®0123456789')).lower()
  text = text.strip() # Remove whitespaces
  text = text.split() # split the text to words
  text =  " ".join( text ) #Activer ce commentaire si vous voulez appliquer une contractions 
  return text

#************************************************* fonctions pour les types de tests *******************************

def replace_contractions(text):
  clean = classic_processing(text)
  return contractions.fix(clean)

#remove stop_words thanks to nltk, the Data must be in LowerCase + contractions.
stops = set(stopwords.words('english'))  
mylist = pd.read_csv("FrequentWords_v160320.csv")
stops2 = mylist['x'].tolist()

def remove_stop_words(text):
  words=token.tokenize(text)
  meaningful_words = [w for w in words if ((w.isalpha()==1) & (w not in stops) & (w not in stops2))]
  text_final =  " ".join(map(str, meaningful_words))
  return text_final

def lemitizeWords(text):
  words=token.tokenize(text)
  listLemma=[]
  for w in words:
    x=lemma.lemmatize(w, pos="v")
    listLemma.append(x)
  return ' '.join(map(str, listLemma))



#La stemmatisation = verbe infinitif
#les données doivent être tokenizées
def stemmatisation(text):
  words=token.tokenize(text)
  stemmed = [porter.stem(w) for w in words]
  return ' '.join(map(str,stemmed))

#************************************************* fonctions pour les types de classifieurs *******************************

def run_RNN(X):
  seed=7
  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  fold = 0
  for train, test in kfold.split(X):
    fold+=1
    
    x_train = X[train]
    y_train = y[train]
    x_test = X[test]
    y_test = y[test]

    model = Sequential()
    model.add(Dense(4, input_dim= X.shape[1], activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
  
    model.fit(x_train,y_train, epochs=10, batch_size=32, verbose= True)
  
    scores = model.evaluate(x_test,y_test, verbose=0)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
  
  return cvscores
    
 
def run_SVM (X):
  seed=7
  k_fold = KFold(n_splits=10, shuffle=True, random_state=seed)
  clf = SVC(C=1, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear', max_iter=-1, probability=False, random_state=None, shrinking=True,tol=0.001, verbose=False) 
  scoring = 'accuracy'
  score = cross_val_score(clf, X, y, cv=k_fold, scoring=scoring)
  return score.mean()



# Load the Data

In [23]:
import pandas as pd
import numpy as np

#load the Data
Data_pos = pd.read_csv('yes_not_clean.csv')
Data_neg = pd.read_csv('no_not_clean.csv',error_bad_lines=False , engine='python')
Neg = Data_neg[0:512] # extraire 512 observations non-INm pour avoir 1 équilibre 

Pos = Data_pos[['text','response']]
Neg = Neg[['text','response']]
Neg['response'].replace({2: 0}, inplace=True) # remplacer la valeur dcu champ respone =2 par 0

df_yes =Pos
df_no = Neg
df_text_yes=df_yes.loc[:,['text', 'response']]
df_text_no=df_no.loc[:,['text', 'response']]
# récupération des X_yes et y_yes
array = df_text_yes.values
X_yes = array[:,0] 
y_yes = array[:,1]

# récupération des X_no et y_no
array = df_text_no.values
X_no=array[:,0] 
y_no = array[:,1]

# concatenation dans X des X_yes et X_no
X=np.concatenate((X_yes, X_no))
y=np.concatenate((y_yes, y_no))

#transformation des y en entier pour la classification
y=y.astype('int')

#transformation des X en list de string
X=np.array(X).tolist()


Skipping line 3522: unexpected end of data


# Text Brut

In [17]:
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X)
X = vectors.toarray()
print ("RNN : texte_brut",run_RNN(X))
print ("SVM : texte_brut ",run_SVM(X))

Epoch 1/10
920/920 [==============================] - 1s 716us/step - loss: 0.6734 - acc: 0.6739
Epoch 2/10
920/920 [==============================] - 0s 140us/step - loss: 0.6072 - acc: 0.9293
Epoch 3/10
920/920 [==============================] - 0s 136us/step - loss: 0.5409 - acc: 0.9543
Epoch 4/10
920/920 [==============================] - 0s 135us/step - loss: 0.4809 - acc: 0.9620
Epoch 5/10
920/920 [==============================] - 0s 144us/step - loss: 0.4263 - acc: 0.9685
Epoch 6/10
920/920 [==============================] - 0s 138us/step - loss: 0.3780 - acc: 0.9707
Epoch 7/10
920/920 [==============================] - 0s 142us/step - loss: 0.3350 - acc: 0.9750
Epoch 8/10
920/920 [==============================] - 0s 141us/step - loss: 0.2975 - acc: 0.9783
Epoch 9/10
920/920 [==============================] - 0s 149us/step - loss: 0.2647 - acc: 0.9804
Epoch 10/10
920/920 [==============================] - 0s 141us/step - loss: 0.2360 - acc: 0.9837
Epoch 1/10
920/920 [=========

#traitement classique text 

In [15]:
list_classic_process = []
for text in X:
  list_classic_process.append(classic_processing(text))

X2 = list_classic_process

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X2)
X2 = vectors.toarray()
print ("RNN : text_traitement_classique ",run_RNN(X2))
print ("SVM : text_traitement_classique ",run_SVM(X2))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
920/920 [==============================] - 0s 368us/step - loss: 0.6786 - acc: 0.8043
Epoch 2/10
920/920 [==============================] - 0s 144us/step - loss: 0.6277 - acc: 0.9554
Epoch 3/10
920/920 [==============================] - 0s 146us/step - loss: 0.5728 - acc: 0.9663
Epoch 4/10
920/920 [==============================] - 0s 153us/step - loss: 0.5167 - acc: 0.9685
Epoch 5/10
920/920 [==============================] - 0s 154us/step - loss: 0.4621 - acc: 0.9750
Epoch 6/10
920/920 [==============================] - 0s 149us/step - loss: 0.4103 - acc: 0.9804
Epoch 7/10
920/920 [==============================] - 0s 145us/step - loss: 0.3633 - acc: 0.9793
Epoch 8/10
920/920 [==============================] - 0s 137us/step - loss: 0.3210 - acc: 0.9815
Epoch 9/10
920/920 [==============================] - 0s 151us/step - loss: 0.2837 - acc: 0.9826
Epoch 10/10
920/920 [===========

# text brut + replace contractions

In [22]:

list_text_brut_replace_contractions = []

for text in X:
  list_text_brut_replace_contractions.append(replace_contractions(text))

X3 = list_text_brut_replace_contractions


vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X3)
X3 = vectors.toarray()
print ("RNN : texte_brut_classic_processing_verbe_infinitif",run_RNN(X3))
print ("SVM : texte_brut_classic_processing_verbe_infinitif",run_SVM(X3))

Epoch 1/10
920/920 [==============================] - 1s 1ms/step - loss: 0.6783 - acc: 0.7478
Epoch 2/10
920/920 [==============================] - 0s 138us/step - loss: 0.6253 - acc: 0.9565
Epoch 3/10
920/920 [==============================] - 0s 136us/step - loss: 0.5689 - acc: 0.9641
Epoch 4/10
920/920 [==============================] - 0s 139us/step - loss: 0.5123 - acc: 0.9696
Epoch 5/10
920/920 [==============================] - 0s 137us/step - loss: 0.4572 - acc: 0.9728
Epoch 6/10
920/920 [==============================] - 0s 137us/step - loss: 0.4053 - acc: 0.9772
Epoch 7/10
920/920 [==============================] - 0s 144us/step - loss: 0.3583 - acc: 0.9793
Epoch 8/10
920/920 [==============================] - 0s 148us/step - loss: 0.3161 - acc: 0.9815
Epoch 9/10
920/920 [==============================] - 0s 138us/step - loss: 0.2792 - acc: 0.9837
Epoch 10/10
920/920 [==============================] - 0s 142us/step - loss: 0.2473 - acc: 0.9848
Epoch 1/10
920/920 [===========

# text + traitement classique + replace contractions

In [24]:

list_text_traité_replace_contractions = []
X4 = list_classic_process

for text in X4:
  list_text_traité_replace_contractions.append(replace_contractions(text))

X5 = list_text_traité_replace_contractions


vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X5)
X5 = vectors.toarray()
print ("RNN : text_traité_replace_contractions",run_RNN(X5))
print ("SVM : list_text_traité_replace_contractions ",run_SVM(X5))

Epoch 1/10
920/920 [==============================] - 2s 2ms/step - loss: 0.6828 - acc: 0.7489
Epoch 2/10
920/920 [==============================] - 0s 143us/step - loss: 0.6433 - acc: 0.8815
Epoch 3/10
920/920 [==============================] - 0s 149us/step - loss: 0.5958 - acc: 0.9239
Epoch 4/10
920/920 [==============================] - 0s 142us/step - loss: 0.5436 - acc: 0.9457
Epoch 5/10
920/920 [==============================] - 0s 145us/step - loss: 0.4897 - acc: 0.9630
Epoch 6/10
920/920 [==============================] - 0s 144us/step - loss: 0.4364 - acc: 0.9685
Epoch 7/10
920/920 [==============================] - 0s 153us/step - loss: 0.3858 - acc: 0.9750
Epoch 8/10
920/920 [==============================] - 0s 147us/step - loss: 0.3397 - acc: 0.9761
Epoch 9/10
920/920 [==============================] - 0s 155us/step - loss: 0.2986 - acc: 0.9793
Epoch 10/10
920/920 [==============================] - 0s 146us/step - loss: 0.2631 - acc: 0.9804
Epoch 1/10
920/920 [===========

# text brut + verb à l'infinitif

In [26]:

list_classic_process_verb_infintif = []

for text in X:
  list_classic_process_verb_infintif.append(stemmatisation(text))

X6 = list_classic_process_verb_infintif

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X6)
X6 = vectors.toarray()
print ("RNN : texte_brut_verbe_infinitif",run_RNN(X6))
print ("SVM : texte_brut_verbe_infinitif",run_SVM(X6))

Epoch 1/10
920/920 [==============================] - 2s 2ms/step - loss: 0.6656 - acc: 0.8337
Epoch 2/10
920/920 [==============================] - 0s 154us/step - loss: 0.5871 - acc: 0.9478
Epoch 3/10
920/920 [==============================] - 0s 139us/step - loss: 0.5143 - acc: 0.9576
Epoch 4/10
920/920 [==============================] - 0s 137us/step - loss: 0.4488 - acc: 0.9674
Epoch 5/10
920/920 [==============================] - 0s 144us/step - loss: 0.3919 - acc: 0.9685
Epoch 6/10
920/920 [==============================] - 0s 141us/step - loss: 0.3424 - acc: 0.9728
Epoch 7/10
920/920 [==============================] - 0s 151us/step - loss: 0.3003 - acc: 0.9750
Epoch 8/10
920/920 [==============================] - 0s 139us/step - loss: 0.2643 - acc: 0.9783
Epoch 9/10
920/920 [==============================] - 0s 147us/step - loss: 0.2340 - acc: 0.9793
Epoch 10/10
920/920 [==============================] - 0s 144us/step - loss: 0.2077 - acc: 0.9848
Epoch 1/10
920/920 [===========

# text brut + traitement classique + verb à l'infinitf

In [27]:
text_traite_verb_infintif = []

for text in list_classic_process:
  text_traite_verb_infintif.append(stemmatisation(text))

X7 = text_traite_verb_infintif

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X7)
X7 = vectors.toarray()
print ("RNN : texte_traite_verbe_infinitif",run_RNN(X7))
print ("SVM : texte_traite_verbe_infinitif",run_SVM(X7))

Epoch 1/10
920/920 [==============================] - 2s 3ms/step - loss: 0.6750 - acc: 0.7554
Epoch 2/10
920/920 [==============================] - 0s 130us/step - loss: 0.6111 - acc: 0.9446
Epoch 3/10
920/920 [==============================] - 0s 130us/step - loss: 0.5451 - acc: 0.9576
Epoch 4/10
920/920 [==============================] - 0s 129us/step - loss: 0.4833 - acc: 0.9641
Epoch 5/10
920/920 [==============================] - 0s 129us/step - loss: 0.4265 - acc: 0.9685
Epoch 6/10
920/920 [==============================] - 0s 130us/step - loss: 0.3756 - acc: 0.9717
Epoch 7/10
920/920 [==============================] - 0s 130us/step - loss: 0.3309 - acc: 0.9750
Epoch 8/10
920/920 [==============================] - 0s 133us/step - loss: 0.2920 - acc: 0.9772
Epoch 9/10
920/920 [==============================] - 0s 135us/step - loss: 0.2586 - acc: 0.9804
Epoch 10/10
920/920 [==============================] - 0s 136us/step - loss: 0.2300 - acc: 0.9826
Epoch 1/10
920/920 [===========

# Text brut + remove stop words

In [28]:
list_remove_stop_words = []

for text in X:
  list_remove_stop_words.append(remove_stop_words(text))

X8 = list_remove_stop_words

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X8)
X8 = vectors.toarray()
print ("RNN : texte_brut_remove_stop_words",run_RNN(X8))
print ("SVM : texte_brut_remove_stop_words",run_SVM(X8))

Epoch 1/10
920/920 [==============================] - 3s 3ms/step - loss: 0.6801 - acc: 0.8065
Epoch 2/10
920/920 [==============================] - 0s 141us/step - loss: 0.6372 - acc: 0.9467
Epoch 3/10
920/920 [==============================] - 0s 148us/step - loss: 0.5897 - acc: 0.9674
Epoch 4/10
920/920 [==============================] - 0s 148us/step - loss: 0.5402 - acc: 0.9728
Epoch 5/10
920/920 [==============================] - 0s 148us/step - loss: 0.4901 - acc: 0.9739
Epoch 6/10
920/920 [==============================] - 0s 144us/step - loss: 0.4409 - acc: 0.9772
Epoch 7/10
920/920 [==============================] - 0s 145us/step - loss: 0.3941 - acc: 0.9815
Epoch 8/10
920/920 [==============================] - 0s 145us/step - loss: 0.3506 - acc: 0.9826
Epoch 9/10
920/920 [==============================] - 0s 143us/step - loss: 0.3113 - acc: 0.9859
Epoch 10/10
920/920 [==============================] - 0s 152us/step - loss: 0.2764 - acc: 0.9880
Epoch 1/10
920/920 [===========

# text brut + traitement classique + remove stop words

In [29]:
text_traite_remove_stop_words = []

for text in list_classic_process:
  text_traite_remove_stop_words.append(remove_stop_words(text))

X9 = text_traite_remove_stop_words

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X9)
X9 = vectors.toarray()
print ("RNN : texte_brut_classic_processing_remove_stop_words",run_RNN(X9))
print ("SVM : texte_brut_classic_processing_remove_stop_words",run_SVM(X9))

Epoch 1/10
920/920 [==============================] - 4s 4ms/step - loss: 0.6814 - acc: 0.7848
Epoch 2/10
920/920 [==============================] - 0s 162us/step - loss: 0.6368 - acc: 0.9424
Epoch 3/10
920/920 [==============================] - 0s 162us/step - loss: 0.5874 - acc: 0.9587
Epoch 4/10
920/920 [==============================] - 0s 170us/step - loss: 0.5374 - acc: 0.9696
Epoch 5/10
920/920 [==============================] - 0s 180us/step - loss: 0.4887 - acc: 0.9750
Epoch 6/10
920/920 [==============================] - 0s 202us/step - loss: 0.4416 - acc: 0.9793
Epoch 7/10
920/920 [==============================] - 0s 176us/step - loss: 0.3977 - acc: 0.9804
Epoch 8/10
920/920 [==============================] - 0s 164us/step - loss: 0.3572 - acc: 0.9837
Epoch 9/10
920/920 [==============================] - 0s 170us/step - loss: 0.3205 - acc: 0.9859
Epoch 10/10
920/920 [==============================] - 0s 170us/step - loss: 0.2872 - acc: 0.9880
Epoch 1/10
920/920 [===========

# text brut + lemmatisation

In [30]:
text_brut_lemmatisation = []


for text in X:
  text_brut_lemmatisation.append(lemitizeWords(text))

X10 = text_brut_lemmatisation

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X10)
X10 = vectors.toarray()
print ("RNN : texte_brut_lemmatisation",run_RNN(X10))
print ("SVM : texte_brut__lemmatisation",run_SVM(X10))

Epoch 1/10
920/920 [==============================] - 4s 5ms/step - loss: 0.6796 - acc: 0.7978
Epoch 2/10
920/920 [==============================] - 0s 162us/step - loss: 0.6348 - acc: 0.9315
Epoch 3/10
920/920 [==============================] - 0s 176us/step - loss: 0.5861 - acc: 0.9337
Epoch 4/10
920/920 [==============================] - 0s 176us/step - loss: 0.5358 - acc: 0.9446
Epoch 5/10
920/920 [==============================] - 0s 168us/step - loss: 0.4862 - acc: 0.9554
Epoch 6/10
920/920 [==============================] - 0s 170us/step - loss: 0.4385 - acc: 0.9663
Epoch 7/10
920/920 [==============================] - 0s 171us/step - loss: 0.3941 - acc: 0.9674
Epoch 8/10
920/920 [==============================] - 0s 180us/step - loss: 0.3535 - acc: 0.9739
Epoch 9/10
920/920 [==============================] - 0s 179us/step - loss: 0.3163 - acc: 0.9739
Epoch 10/10
920/920 [==============================] - 0s 182us/step - loss: 0.2832 - acc: 0.9783
Epoch 1/10
920/920 [===========

# text brut + traitement classique + lemmatisation

In [31]:
text_traite_lemmatisation = []


for text in list_classic_process:
  text_traite_lemmatisation.append(lemitizeWords(text))

X12 = text_traite_lemmatisation

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X12)
X12 = vectors.toarray()
print ("RNN : texte_traite_lemmatisation",run_RNN(X12))
print ("SVM : texte_traite_lemmatisation",run_SVM(X12))

Epoch 1/10
920/920 [==============================] - 5s 5ms/step - loss: 0.6809 - acc: 0.6587
Epoch 2/10
920/920 [==============================] - 0s 201us/step - loss: 0.6375 - acc: 0.7685
Epoch 3/10
920/920 [==============================] - 0s 204us/step - loss: 0.5902 - acc: 0.8609
Epoch 4/10
920/920 [==============================] - 0s 206us/step - loss: 0.5444 - acc: 0.8946
Epoch 5/10
920/920 [==============================] - 0s 191us/step - loss: 0.4995 - acc: 0.9185
Epoch 6/10
920/920 [==============================] - 0s 200us/step - loss: 0.4562 - acc: 0.9457
Epoch 7/10
920/920 [==============================] - 0s 209us/step - loss: 0.4147 - acc: 0.9609
Epoch 8/10
920/920 [==============================] - 0s 202us/step - loss: 0.3753 - acc: 0.9685
Epoch 9/10
920/920 [==============================] - 0s 204us/step - loss: 0.3385 - acc: 0.9728
Epoch 10/10
920/920 [==============================] - 0s 185us/step - loss: 0.3044 - acc: 0.9761
Epoch 1/10
920/920 [===========